In [1]:
import pandas as pd
import datetime
from datetime import date

from keras.models import load_model
model = load_model('model_combined_1.h5')


from scripts.get_google_search import get_google_search
from scripts.get_news_sentiment import get_news_sentiment
from scripts.get_tweet_sentiment import get_tweet_sentiment

# target, the stock prices
from scripts.get_stock_price import get_stock_prices

2022-12-07 20:50:28.999897: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 20:50:28.999925: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-07 20:50:30.071432: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-07 20:50:30.071522: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-07 20:50:30.071542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [2]:
# X_train_stocks should contain ["open", "high", "low", "close", "adjusted_close", "volume"], shape is (batch_size, 30, 6)
# X_train_sentiments should contain ["news_count", "news_sentiment", "tweet_count", "tweet_sentiment"]], shape is (batch_size, 30, 4)
stock_ticker = 'A'

In [3]:
# pull stocks price data from API
df_stocks = get_stock_prices(stock_ticker)

# get the right column names
X_train_stocks = df_stocks.rename(columns = {'Open':'open', 
                                             'High':'high', 
                                             'Low':'low',
                                             'Close':'close', 
                                             'Adj Close':'adjusted_close',
                                             'Volume':'volume'})

X_train_stocks

[*********************100%***********************]  1 of 1 completed


,open,high,low,close,adjusted_close,volume
Date,,,,,,
2022-11-07,136.850006,138.080002,135.559998,138.029999,138.029999,1213600
2022-11-08,138.500000,140.690002,136.600006,138.750000,138.750000,1028100
2022-11-09,138.309998,139.419998,136.660004,137.000000,137.000000,971200
2022-11-10,142.869995,146.720001,142.529999,146.300003,146.300003,1591900
2022-11-11,147.119995,149.740005,146.350006,148.309998,148.309998,1227500
2022-11-14,149.009995,149.710007,146.300003,146.380005,146.380005,1220300
2022-11-15,148.270004,149.460007,146.500000,148.000000,148.000000,1299200
2022-11-16,148.190002,148.750000,146.350006,147.119995,147.119995,1476700
2022-11-17,145.330002,145.559998,142.160004,144.440002,144.440002,1465000


In [4]:
# fill missing price data (for the weekends) with the last known price for the stock 
date_range = pd.date_range((date.today() - datetime.timedelta(days=29)),
                           date.today())

# first replace with zeros
X_train_stocks.index = pd.DatetimeIndex(X_train_stocks.index)
X_train_stocks = X_train_stocks.reindex(date_range, fill_value=0)

# then replace zeros with the values above them 
# (implying that the previous day's price is the best predictor for the next day)
def replace_missing_with_one_above(column): 
    for index, number in enumerate(column):
        if column[index] != 0: 
            if number == 0:
                 column[index] = column[index-1]
        return column

X_train_stocks.apply(replace_missing_with_one_above)

X_train_stocks.head()

,open,high,low,close,adjusted_close,volume
2022-11-08,138.500000,140.690002,136.600006,138.750000,138.750000,1028100
2022-11-09,138.309998,139.419998,136.660004,137.000000,137.000000,971200
2022-11-10,142.869995,146.720001,142.529999,146.300003,146.300003,1591900
2022-11-11,147.119995,149.740005,146.350006,148.309998,148.309998,1227500
2022-11-12,0.000000,0.000000,0.000000,0.000000,0.000000,0


In [5]:
X_train_stocks.shape

(30, 6)

In [ ]:
api_key = '6373e059e5d049.13587213'

news = get_news_sentiment(stock_ticker, api_key = api_key)
tweets = get_tweet_sentiment(stock_ticker, api_key = api_key)
google = get_google_search(stock_ticker).drop(['isPartial'], axis = 1).reset_index()
google = google.rename(columns={google.columns[1]: 'google_count', google.columns[0]: 'Date'})


In [6]:
X_train_sentiments['google_count'] = X_train_sentiments['google_count'].fillna(0)

,Date,google_count
0,2022-11-08,99
1,2022-11-09,98
2,2022-11-10,100
3,2022-11-11,91
4,2022-11-12,91
5,2022-11-13,92
6,2022-11-14,93
7,2022-11-15,93
8,2022-11-16,93
9,2022-11-17,97


In [9]:
merged_news_tweets = pd.merge(news, tweets, 
                           left_on='Date', 
                           right_on='Date', 
                           how='left')
merged_features = pd.merge(merged_news_tweets, 
                           google,
                          left_on = 'Date', 
                          right_on = 'Date', 
                          how = 'left')
merged_features = merged_features.set_index('Date')

merged_features

,News_Count,News_Sentiment,Tweet_Count,Tweet_Sentiment,google_count
Date,,,,,
2022-12-07,2,0.9935,17,0.0000,NaN
2022-12-06,1,0.9990,9,0.0733,NaN
2022-12-05,1,0.9950,9,0.0424,NaN
2022-11-30,1,0.9940,11,0.0000,89.0
2022-11-29,2,0.9935,13,0.0622,92.0
2022-11-28,2,0.9985,8,0.0000,90.0
2022-11-22,5,0.9824,41,0.0248,83.0
2022-11-21,7,0.7826,53,-0.0137,80.0
2022-11-20,2,0.5060,52,-0.0061,91.0


In [10]:
X_train_sentiments = merged_features.rename(columns= {'News_Count':'news_count', 
                                                         'News_Sentiment': 'news_sentiment', 
                                                         'Tweet_Count': 'tweet_count', 
                                                         'Tweet_Sentiment': 'tweet_sentiment', 
                                                     'google_count': 'google_count'})
# X_train_sentiments should contain ["news_count", "news_sentiment", "tweet_count", "tweet_sentiment"]], shape is (batch_size, 30, 4)
X_train_sentiments.head()

,news_count,news_sentiment,tweet_count,tweet_sentiment,google_count
Date,,,,,
2022-12-07,2,0.9935,17,0.0000,NaN
2022-12-06,1,0.9990,9,0.0733,NaN
2022-12-05,1,0.9950,9,0.0424,NaN
2022-11-30,1,0.9940,11,0.0000,89.0
2022-11-29,2,0.9935,13,0.0622,92.0


In [ ]:
# fill missing price data (for the weekends) with the last known price for the stock 
date_range = pd.date_range((date.today() - datetime.timedelta(days=29)),
                           date.today())

# first replace with zeros
X_train_sentiments.index = pd.DatetimeIndex(X_train_sentiments.index)
X_train_sentiments = X_train_sentiments.reindex(date_range, fill_value=0)
X_train_sentiments

In [33]:
X_train_sentiments['google_count'] = X_train_sentiments['google_count'].fillna(0)
X_train_sentiments

,news_count,news_sentiment,tweet_count,tweet_sentiment,google_count
2022-11-08,2,0.9910,14,0.0000,98.0
2022-11-09,3,0.9933,7,0.0706,99.0
2022-11-10,1,0.9960,19,0.0454,100.0
2022-11-11,0,0.0000,0,0.0000,0.0
2022-11-12,0,0.0000,0,0.0000,0.0
2022-11-13,0,0.0000,0,0.0000,0.0
2022-11-14,5,0.9734,40,0.0351,95.0
2022-11-15,1,-0.6910,12,0.0337,95.0
2022-11-16,5,0.7574,9,0.0555,93.0
2022-11-17,3,0.4740,18,0.0000,96.0


In [34]:
# then replace zeros with the values above them 
# (implying that the previous day's price is the best predictor for the next day)
def replace_missing_with_one_above(column): 
    for index, number in enumerate(column):
        if column[index] != 0: 
            if number == 0:
                 column[index] = column[index-1]
        return column

X_train_sentiments.apply(replace_missing_with_one_above)

X_train_sentiments.head()

,news_count,news_sentiment,tweet_count,tweet_sentiment,google_count
2022-11-08,2,0.9910,14,0.0000,98.0
2022-11-09,3,0.9933,7,0.0706,99.0
2022-11-10,1,0.9960,19,0.0454,100.0
2022-11-11,0,0.0000,0,0.0000,0.0
2022-11-12,0,0.0000,0,0.0000,0.0


In [35]:
X_train_sentiments.shape

(30, 5)

In [36]:
X_train_stocks.shape

(30, 6)

In [77]:
  

# model = load_model("model_combined_1.h5")
stock_prediction_14 = model.predict([X_train_stocks, X_train_sentiments])[0].tolist()
stock_prediction_14
# X_train_stocks should contain ["open", "high", "low", "close", "adjusted_close", "volume"], shape is (batch_size, 30, 6)
# X_train_sentiments should contain ["news_count", "news_sentiment", "tweet_count", "tweet_sentiment"]], shape is (batch_size, 30, 4)

# Data should be for the last 30 days

1/1 [==============================] - 1s 1s/step


[43.39475631713867,
 45.870330810546875,
 39.927364349365234,
 45.80978775024414,
 46.66742706298828,
 41.38005065917969,
 37.363338470458984,
 47.07857894897461,
 41.821739196777344,
 46.95650863647461,
 49.27693176269531,
 47.2007942199707,
 45.424041748046875,
 41.945289611816406]